<a href="https://colab.research.google.com/github/r-alfonso/powerbi_tableau/blob/main/tweets_powerbi_tableau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Librerias

In [ ]:
import tweepy

In [ ]:
import pandas as pd

In [ ]:
import pyodbc

## Conexión con la API

### Credenciales:

##### Información sobre la API y como obtener las credenciales: https://developer.twitter.com/en/docs/twitter-api

Lo ideal es tenerlas en un archivo separado

In [ ]:
api_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
api_secret_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
access_token = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
access_token_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

### Autentificación

In [ ]:
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)

### Enlace API

In [ ]:
api = tweepy.API(auth,
                 wait_on_rate_limit = True,
                 wait_on_rate_limit_notify = True)

## Busqueda de Tweets en español

### Criterios de la búsqueda

In [ ]:
content_search = ('#PowerBI OR #Tableau -filter:retweets')

El operador OR debe quedar dentro de la cadena y en mayuscula. No modificar por 'xxx' OR 'yyy' pues solo devolveria el primero. Por otro lado, -filter:retweets excluye los retweets.

In [ ]:
id_tweet_since = '1389006799271029563'

El Tweet ID se utiliza para indicar la fecha (el parámetro since_id por ejemplo, indica una fecha superior o igual al del id del tweet especificado), pues cada id de twitter "lleva impreso" la fecha y hora en la cual se publicó (en este caso el id '1389006799271029563' corresponde al lunes 03-05-2021 00:00:02 GMT. Para ampliar este tema y obtener la fecha de un ID puedes consulta el repositorio: https://github.com/oduwsdl/tweetedat

Otro método puede ser directamente indicar la fecha mediante el parámetro proporcionado para dicho caso (https://docs.tweepy.org/en/latest/index.html), dado que en algunas ocasiones este metodo me generó errores, prefiero el uso del id.

In [ ]:
sample = 2500

In [ ]:
language_code = 'es'

### Búsqueda de Tweets y Creación del Dataframe

In [ ]:
tweets = tweepy.Cursor(api.search, q = content_search, lang = language_code, tweet_mode = 'extended', since_id = id_tweet_since).items(sample)
df = pd.DataFrame(columns = ['id_user','username','id_tweet','tweet','created_at','user_location'])
for tweet in tweets:
    df = df.append({'id_user':tweet.user.id,'username':tweet.user.screen_name,'id_tweet':tweet.id,'tweet':tweet.full_text,'created_at':tweet.created_at,'user_location':tweet.user.location}, ignore_index = True)

Hasta acá se prodría exportar el dataframe a un archivo .csv //df.to_csv('nombre_del_archivo.csv',index = False)// pero podemos cargarlo directamente a un database.

## Creación de tabla en sql server y carga del Dataframe

### Conexión con la base

In [ ]:
conexion_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=;DATABASE=;UID=r'.\SQLEXPRESS';PWD=;Trusted_Connection=yes;"
conexion = pyodbc.connect(conexion_string, autocommit=True)

### Creación de tabla

In [ ]:
tweets_table= 'CREATE TABLE tweets_table(id_user nvarchar(255),username nvarchar(255),id_tweet nvarchar(255),tweet nvarchar(MAX),created_at nvarchar(255),user_location nvarchar(255))'
cursor = conexion.cursor()
cursor.execute(tweets_table)

### Carga del Dataframe

In [ ]:
for index, row in df.iterrows(): 
    conexion.execute("INSERT INTO tweets_table (id_user, username, id_tweet, tweet, created_at, user_location) values(?,?,?,?,?,?)", row['id_user'], row['username'], row['id_tweet'], row['tweet'], row['created_at'], row['user_location'])

### Finalización de la Conexión

In [ ]:
conexion.close()
print ('Listo el pollo')